# Generate and save ECODE responses, protocols, and features

In [ ]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import model
import evaluator
import time
import neuron
import plotting
import MEAutility as mu
from pprint import pprint
import numpy as np

import sys
import shutil

from pathlib import Path
import os

%matplotlib widget

## 1) Generate and run ecode protocols

In [ ]:
sys.path.append('../ecode/')

In [ ]:
from ecode import generate_ecode_protocols, compute_rheobase_for_model, run_ecode_protocols, \
        save_intracellular_responses, save_extracellular_template

In [ ]:
probe_type = "planar" # linear 
model_name = 'hay' # "hallerman"
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
# output folder for data
output_folder = Path(f"../data/{model_name}_ecode_probe_{probe_type}")

In [ ]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".").absolute()
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")
    compiled_folder = Path("./x86_64")
    if compiled_folder.is_dir():
        shutil.rmtree(compiled_folder)
    shutil.copytree(model_folder / "x86_64", compiled_folder)

In [ ]:
cell = model.create(model_name=model_name, release=True)

probe = model.define_electrode(probe_type=probe_type)
# probe = None

param_names = [param.name for param in cell.params.values() if not param.frozen]
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)

In [ ]:
rheobase, rheo_protocols, rheo_responses = compute_rheobase_for_model(cell, sim=sim, step_min=0.34)

In [ ]:
ecode_protocols = generate_ecode_protocols(rheobase_current=rheobase, record_extra=True,
                                           protocols_with_lfp="IDrest_300")

In [ ]:
responses_dict = run_ecode_protocols(protocols=ecode_protocols, cell=cell, sim=sim, 
                                     resample_rate_khz=40)

In [ ]:
save_intracellular_responses(responses_dict=responses_dict, output_folder=output_folder)

In [ ]:
eap, locations = save_extracellular_template(responses=responses_dict["IDrest"], 
                                             protocols=ecode_protocols, protocol_name="IDrest",
                                             probe=probe, output_folder=output_folder, sweep_id=10, 
                                             resample_rate_khz=20, fcut=[300, 6000],
                                             filt_type="filtfilt")

In [ ]:
ax = mu.plot_mea_recording(eap, probe)

In [ ]:
plt.figure()
_ = plt.plot(eap.T, "k")

## 2) Generate feature and protocols with BluePyEfe

In [ ]:
sys.path.append('../efeatures_extraction')

In [ ]:
efeatures_output_directory = Path(f"../data/{model_name}_ecode_probe_{probe_type}/efeatures")

In [ ]:
from bluepyefe.extract import read_recordings, extract_efeatures_at_targets, compute_rheobase,\
    group_efeatures, create_feature_protocol_files, convert_legacy_targets
from bluepyefe.plotting import plot_all_recordings_efeatures

from extraction_tools import build_model_metadata, model_csv_reader, get_targets, ecodes_model_timings

In [ ]:
files_metadata = build_model_metadata(cell_id=model_name, ephys_dir=output_folder)
pprint(files_metadata[model_name])

In [ ]:
cells = read_recordings(
    files_metadata=files_metadata,
    recording_reader=model_csv_reader
)

In [ ]:
# define target features for different protocols
targets = get_targets(ecodes_model_timings)
targets = convert_legacy_targets(targets)
pprint(targets)

In [ ]:
t_start = time.time()
extract_efeatures_at_targets(
    cells, 
    targets,
)
t_stop = time.time()
print(f"Elapsed time {t_stop - t_start}")

In [ ]:
compute_rheobase(
    cells, 
    protocols_rheobase=['IDthres']
)

In [ ]:
protocols = group_efeatures(cells, targets, use_global_rheobase=True)

In [ ]:
efeatures, protocol_definitions, current = create_feature_protocol_files(
    cells,
    protocols,
    output_directory=efeatures_output_directory,
    threshold_nvalue_save=1,
    write_files=True,
)

## 3) Convert to BPO format and append extra features

In [ ]:
from extraction_tools import convert_to_bpo_format, append_extrafeatures_to_json, compute_extra_features

In [ ]:
protocols_of_interest = ["IDrest_150", "IDrest_250", "IDrest_300", "IV_-100", "IV_-20", "APWaveform_260"]

in_protocol_path = efeatures_output_directory / "protocols.json"
in_efeatures_path = efeatures_output_directory / "features.json"

out_protocol_path = efeatures_output_directory / "protocols_BPO.json"
out_efeatures_path = efeatures_output_directory / "features_BPO.json"

In [ ]:
protocols_dict, efeatures_dict = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                       out_protocol_path, out_efeatures_path, 
                                                       protocols_of_interest=protocols_of_interest, 
                                                       std_from_mean=0.2)

In [ ]:
efeatures_dict

In [ ]:
# append MEA.LFP features
eap = np.load(output_folder / "extracellular" / "template.npy")
fs = np.load(output_folder / "extracellular" / "fs.npy")

In [ ]:
extra_features = compute_extra_features(eap, fs, upsample=10)

In [ ]:
pprint(extra_features)

In [ ]:
efeatures_dict = append_extrafeatures_to_json(extra_features, protocol_name="IDrest_300",
                                              efeatures_path=out_efeatures_path)

In [ ]:
json.dump(probe.info, (efeatures_output_directory / "probe_BPO.json").open("w"))

In [ ]:
# plot one extra features
f = plotting.plot_feature_map_w_colorbar(extra_features["peak_trough_ratio"], probe, 
                                         feature_name="peak_trough_ratio", label="time (s)")

In [ ]:
ax = mu.plot_mea_recording(eap / np.max(np.abs(eap), axis=1, keepdims=True), probe)